# Test de Data Leakage en RandomForestFeaturesTransformer

Este notebook reproduce el código del RandomForestFeaturesTransformer y evalúa si hay data leakage calculando e imprimiendo:
- Ganancia en el conjunto de entrenamiento
- Ganancia en el conjunto de validación

Si la ganancia de validación es muy alta o similar a la de entrenamiento (sin un modelo adicional entrenado), puede indicar data leakage.


In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.base import BaseEstimator, TransformerMixin
import logging
from flaml.default import preprocess_and_suggest_hyperparams

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)


## 1. Definir constantes y parámetros


In [ ]:
# Constantes de ganancia
GANANCIA_ACIERTO = 273000
COSTO_ESTIMULO = 7000

# Meses
training_months = [202101, 202102]
eval_month = 202104

# Columnas a excluir
exclude_cols = ["numero_de_cliente", "label", "weight", "clase_ternaria"]


## 2. Implementación del RandomForestFeaturesTransformer


In [ ]:
class BaseTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        if "clase_ternaria" in X.columns:
            raise ValueError("La columna 'clase_ternaria' no debe estar en el dataset")
        X_transformed = self._transform(X)
        return X_transformed


class RandomForestFeaturesTransformer(BaseTransformer):
    def __init__(self, exclude_cols=["numero_de_cliente", "label", "weight", "clase_ternaria"], 
                 n_estimators=20, num_leaves=16, min_data_in_leaf=100, 
                 feature_fraction_bynode=0.2, training_months=[], use_zero_shot=False):
        self.exclude_cols = exclude_cols
        self.training_months = training_months  
        self.n_estimators = n_estimators
        self.use_zero_shot = use_zero_shot
        self.lgb_params = {
            "num_iterations": n_estimators,
            "num_leaves": num_leaves,
            "min_data_in_leaf": min_data_in_leaf,
            "feature_fraction_bynode": feature_fraction_bynode,
            "boosting": "rf",
            "bagging_fraction": (1.0 - 1.0 / np.exp(1.0)),
            "bagging_freq": 1,
            "feature_fraction": 1.0,
            "max_bin": 31,
            "objective": "binary",
            "first_metric_only": True,
            "boost_from_average": True,
            "feature_pre_filter": False,
            "force_row_wise": True,
            "verbosity": -100,
            "max_depth": -1,
            "min_gain_to_split": 0.0,
            "min_sum_hessian_in_leaf": 0.001,
            "lambda_l1": 0.0,
            "lambda_l2": 0.0,
            "pos_bagging_fraction": 1.0,
            "neg_bagging_fraction": 1.0,
            "is_unbalance": False,
            "scale_pos_weight": 1.0,
            "drop_rate": 0.1,
            "max_drop": 50,
            "skip_drop": 0.5,
            "extra_trees": False
        }
    
    def fit(self, X, y=None):
        logger.info(f"Entrenando RandomForestFeaturesTransformer con {self.training_months} meses")
        X = X.copy()
        X_train = X.loc[X["foto_mes"].isin(self.training_months)]
        y = X_train["label"]
        self.keep_cols = [col for col in X_train.columns if col not in self.exclude_cols]
        X_train = X_train[self.keep_cols]
        self.columns_ = X_train.columns
        
        if self.use_zero_shot:
            (
                hp,
                estimator_class,
                X_transformed,
                y_transformed,
                feature_transformer,
                label_transformer,
            ) = preprocess_and_suggest_hyperparams("classification", X, y, "rf")
            self.lgb_params.update({
                "num_iterations": hp["n_estimators"], 
                "num_leaves": hp["max_leaf_nodes"], 
                "feature_fraction": hp["max_features"]
            })
        
        dtrain = lgb.Dataset(
            data=X_train.values,
            label=y,
            free_raw_data=False
        )
        self.model_ = lgb.train(params=self.lgb_params, train_set=dtrain)
        logger.info("RandomForestFeaturesTransformer entrenado")
        
        return self
    
    def _transform(self, X):
        X = X.copy()
        extra_cols = set(X.columns) - set(self.keep_cols)
        extra_cols = X[list(extra_cols)]
        X = X[self.keep_cols]

        prediccion = self.model_.predict(X.values, pred_leaf=True)
        prediccion = np.array(prediccion, dtype=int)

        n_obs, n_trees = prediccion.shape
        logger.info(f"Generando {n_trees} árboles de features...")
        new_cols = {}
        for tree in range(n_trees):
            leaves = np.unique(prediccion[:, tree])
            for leaf in leaves:
                varname = f"rf_{tree + 1:03d}_{leaf:03d}"
                new_cols[varname] = (prediccion[:, tree] == leaf).astype(int)
        
        if new_cols:
            logger.info(f"Se generaron {len(new_cols)} nuevas columnas")
            new_cols_df = pd.DataFrame(new_cols, index=X.index)
            X = pd.concat([X, new_cols_df, extra_cols], axis=1)
        else:
            logger.info("No se generaron nuevas columnas")
            X = pd.concat([X, extra_cols], axis=1)
        
        return X


## 3. Función para calcular ganancia


In [ ]:
def gan_eval(y_pred, weight, window=2001):
    """
    Evalúa la ganancia máxima usando una media móvil centrada con ventana de tamaño `window`.
    Retorna el mejor valor encontrado.
    """
    ganancia = np.where(weight == 1.00002, GANANCIA_ACIERTO, 0) - np.where(weight < 1.00002, COSTO_ESTIMULO, 0)
    ganancia = ganancia[np.argsort(y_pred)[::-1]]
    ganancia = np.cumsum(ganancia)
    
    opt_sends = np.argmax(ganancia)
    if opt_sends - (window-1)/2 < 0:
        min_sends = 0
    else:
        min_sends = int(opt_sends - (window-1)/2)
    if opt_sends + (window-1)/2 > len(ganancia):
        max_sends = len(ganancia)
    else:
        max_sends = int(opt_sends + (window-1)/2)
    
    mean_ganancia = np.mean(ganancia[min_sends:max_sends])
    
    # Calcula la media móvil centrada con la ventana especificada
    ventana = window
    pad = ventana // 2
    ganancia_padded = np.pad(ganancia, (pad, ventana - pad - 1), mode='edge')
    # Calcula la media móvil centrada
    medias_moviles = np.convolve(ganancia_padded, np.ones(ventana)/ventana, mode='valid')

    # Obtiene el máximo de la media móvil centrada
    mejor_ganancia = np.max(medias_moviles)
    
    return mejor_ganancia, mean_ganancia


## 4. Cargar datos


In [ ]:
# Cargar dataset
df = pd.read_csv('data/competencia_01_target.csv')
df = df.drop(columns=["mprestamos_personales", "cprestamos_personales"])

# Crear pesos y labels
weight = {"BAJA+1": 1, "BAJA+2": 1.00002, "CONTINUA": 1}
df["weight"] = df["clase_ternaria"].map(weight)
df["label"] = ((df["clase_ternaria"] == "BAJA+2") | (df["clase_ternaria"] == "BAJA+1")).astype(int)

print(f"Dataset cargado: {df.shape}")
print(f"Meses disponibles: {sorted(df['foto_mes'].unique())}")


## 5. Preparar datos para entrenamiento y validación


In [ ]:
# Filtrar solo los meses de entrenamiento y validación
df_work = df[df["foto_mes"].isin(training_months + [eval_month])].copy()

print(f"\nDatos de trabajo: {df_work.shape}")
print(f"Meses de entrenamiento: {training_months}")
print(f"Mes de validación: {eval_month}")
print(f"\nDistribución por mes:")
print(df_work.groupby('foto_mes').size())
print(f"\nDistribución de labels:")
print(df_work.groupby(['foto_mes', 'label']).size())


## 6. Entrenar el RandomForestFeaturesTransformer

**IMPORTANTE:** Este transformer entrena un Random Forest internamente usando SOLO los meses de training_months.
Si hay data leakage, las features generadas tendrán información del futuro.


In [ ]:
# Crear el transformer
rf_transformer = RandomForestFeaturesTransformer(
    exclude_cols=exclude_cols,
    n_estimators=20,
    num_leaves=16,
    min_data_in_leaf=100,
    feature_fraction_bynode=0.2,
    training_months=training_months,
    use_zero_shot=False
)

# Hacer fit del transformer con TODOS los datos (train + eval)
# El transformer internamente filtrará por training_months
rf_transformer.fit(df_work)

print("\nTransformer entrenado exitosamente")


## 7. Transformar datos y crear features

Ahora aplicamos el transformer a los datos de entrenamiento y validación.


In [ ]:
# Transformar todos los datos
df_transformed = rf_transformer.transform(df_work)

print(f"\nDatos transformados: {df_transformed.shape}")
print(f"Columnas originales: {df_work.shape[1]}")
print(f"Nuevas columnas agregadas: {df_transformed.shape[1] - df_work.shape[1]}")

# Identificar las nuevas columnas de features RF
rf_features = [col for col in df_transformed.columns if col.startswith('rf_')]
print(f"\nTotal de features RF creadas: {len(rf_features)}")
print(f"Ejemplos de features RF: {rf_features[:5]}")


## 8. Separar train y validación


In [ ]:
# Separar train y validación
df_train = df_transformed[df_transformed["foto_mes"].isin(training_months)].copy()
df_val = df_transformed[df_transformed["foto_mes"] == eval_month].copy()

print(f"Train: {df_train.shape}")
print(f"Validación: {df_val.shape}")

# Extraer labels y weights
y_train = df_train["label"].values
w_train = df_train["weight"].values

y_val = df_val["label"].values
w_val = df_val["weight"].values


## 9. Calcular "predicciones" usando solo las features RF

**PRUEBA DE DATA LEAKAGE:**

Si las features del Random Forest contienen data leakage, deberían tener poder predictivo por sí solas, incluso SIN entrenar un modelo adicional.

Vamos a calcular una "predicción" simple sumando los valores de las features RF (todas son 0 o 1). Si hay leakage, esta suma debería correlacionar fuertemente con el target.


In [ ]:
# Usar las features RF como "score" de predicción
# Simplemente sumamos todas las features RF (que son 0/1)
# Si hay leakage, esta suma debería tener poder predictivo

rf_score_train = df_train[rf_features].sum(axis=1).values
rf_score_val = df_val[rf_features].sum(axis=1).values

print("\n=== ANÁLISIS DESCRIPTIVO ===")
print(f"\nScore RF Train - Min: {rf_score_train.min()}, Max: {rf_score_train.max()}, Mean: {rf_score_train.mean():.2f}")
print(f"Score RF Val - Min: {rf_score_val.min()}, Max: {rf_score_val.max()}, Mean: {rf_score_val.mean():.2f}")

# Correlación con el target
from scipy.stats import pearsonr, spearmanr

corr_train_pearson, p_train_pearson = pearsonr(rf_score_train, y_train)
corr_val_pearson, p_val_pearson = pearsonr(rf_score_val, y_val)

corr_train_spearman, p_train_spearman = spearmanr(rf_score_train, y_train)
corr_val_spearman, p_val_spearman = spearmanr(rf_score_val, y_val)

print("\n=== CORRELACIÓN CON EL TARGET ===")
print(f"\nTrain - Pearson: {corr_train_pearson:.4f} (p={p_train_pearson:.4e})")
print(f"Val - Pearson: {corr_val_pearson:.4f} (p={p_val_pearson:.4e})")
print(f"\nTrain - Spearman: {corr_train_spearman:.4f} (p={p_train_spearman:.4e})")
print(f"Val - Spearman: {corr_val_spearman:.4f} (p={p_val_spearman:.4e})")


## 10. Calcular ganancia usando las features RF directamente

**PRUEBA CLAVE DE DATA LEAKAGE:**

Vamos a calcular la ganancia usando SOLO el score RF (suma de features RF) sin entrenar ningún modelo adicional.

- Si obtenemos una ganancia significativa en VALIDACIÓN sin haber entrenado un modelo predictivo, eso es una SEÑAL FUERTE de data leakage.
- Las features RF por sí solas NO deberían tener poder predictivo en datos futuros, a menos que contengan información del futuro.


In [ ]:
# Calcular ganancia en TRAIN usando solo las features RF
ganancia_train, mean_ganancia_train = gan_eval(rf_score_train, w_train, window=2001)

# Calcular ganancia en VALIDACIÓN usando solo las features RF
ganancia_val, mean_ganancia_val = gan_eval(rf_score_val, w_val, window=2001)

print("\n" + "="*80)
print("RESULTADOS DE GANANCIA - PRUEBA DE DATA LEAKAGE")
print("="*80)
print("\n⚠️  IMPORTANTE: Estas ganancias se calculan usando SOLO las features del Random Forest")
print("    SIN entrenar ningún modelo adicional. Si las ganancias son altas, especialmente")
print("    en validación, es una señal de DATA LEAKAGE.\n")

print(f"\n{'='*80}")
print(f"GANANCIA EN ENTRENAMIENTO (meses {training_months}):")
print(f"{'='*80}")
print(f"  - Ganancia máxima (media móvil): ${ganancia_train:,.2f}")
print(f"  - Ganancia media: ${mean_ganancia_train:,.2f}")

print(f"\n{'='*80}")
print(f"GANANCIA EN VALIDACIÓN (mes {eval_month}):")
print(f"{'='*80}")
print(f"  - Ganancia máxima (media móvil): ${ganancia_val:,.2f}")
print(f"  - Ganancia media: ${mean_ganancia_val:,.2f}")

print(f"\n{'='*80}")
print(f"ANÁLISIS DE LEAKAGE:")
print(f"{'='*80}")

# Ratio de ganancia val/train
if ganancia_train > 0:
    ratio_ganancia = ganancia_val / ganancia_train
    print(f"  - Ratio Ganancia Val/Train: {ratio_ganancia:.4f}")
    
    if ratio_ganancia > 0.8:
        print("\n  🚨 ALERTA ROJA: Ganancia de validación muy alta respecto a train!")
        print("     Esto es una SEÑAL FUERTE de data leakage.")
    elif ratio_ganancia > 0.5:
        print("\n  ⚠️  ADVERTENCIA: Ganancia de validación sospechosamente alta.")
        print("     Posible data leakage.")
    elif ganancia_val > 50000000:  # 50M
        print("\n  ⚠️  ADVERTENCIA: Ganancia de validación absoluta muy alta.")
        print("     Posible data leakage.")
    else:
        print("\n  ✅ Las ganancias parecen razonables para features sin modelo adicional.")
        print("     No hay señales evidentes de data leakage en esta prueba.")
else:
    print("  - No se puede calcular el ratio (ganancia train = 0)")
    if ganancia_val > 50000000:  # 50M
        print("\n  🚨 ALERTA: Ganancia de validación alta con ganancia de train baja/cero.")
        print("     SEÑAL FUERTE de data leakage.")

print(f"\n{'='*80}\n")


## 11. Análisis adicional: Distribución de features RF por target

Vamos a ver si las features RF tienen una distribución diferente entre BAJA+2 y CONTINUA.


In [ ]:
print("\n=== DISTRIBUCIÓN DE SCORE RF POR TARGET ===")

# Train
print("\nENTRENAMIENTO:")
print(f"Score RF promedio para target=0 (CONTINUA): {df_train[df_train['label']==0][rf_features].sum(axis=1).mean():.2f}")
print(f"Score RF promedio para target=1 (BAJA+1/+2): {df_train[df_train['label']==1][rf_features].sum(axis=1).mean():.2f}")

# Val
print("\nVALIDACIÓN:")
print(f"Score RF promedio para target=0 (CONTINUA): {df_val[df_val['label']==0][rf_features].sum(axis=1).mean():.2f}")
print(f"Score RF promedio para target=1 (BAJA+1/+2): {df_val[df_val['label']==1][rf_features].sum(axis=1).mean():.2f}")

# T-test para ver si hay diferencia significativa
from scipy.stats import ttest_ind

score_train_0 = df_train[df_train['label']==0][rf_features].sum(axis=1)
score_train_1 = df_train[df_train['label']==1][rf_features].sum(axis=1)
t_stat_train, p_value_train = ttest_ind(score_train_0, score_train_1)

score_val_0 = df_val[df_val['label']==0][rf_features].sum(axis=1)
score_val_1 = df_val[df_val['label']==1][rf_features].sum(axis=1)
t_stat_val, p_value_val = ttest_ind(score_val_0, score_val_1)

print("\n=== T-TEST (diferencia entre target=0 y target=1) ===")
print(f"\nTrain: t-statistic={t_stat_train:.4f}, p-value={p_value_train:.4e}")
print(f"Val: t-statistic={t_stat_val:.4f}, p-value={p_value_val:.4e}")

if p_value_val < 0.001:
    print("\n⚠️  Las features RF muestran diferencias SIGNIFICATIVAS entre target=0 y target=1")
    print("   en el conjunto de validación. Esto podría indicar:")
    print("   1. Las features capturan patrones útiles (bueno)")
    print("   2. O contienen data leakage (malo)")
    print("   La ganancia calculada arriba ayuda a determinar cuál es el caso.")


## 12. Conclusiones

### ¿Cómo interpretar los resultados?

**SIN Data Leakage esperaríamos:**
- Ganancia de validación BAJA o cercana a 0 usando solo las features RF
- Baja correlación entre el score RF y el target en validación
- Las features RF por sí solas NO deberían predecir bien en meses futuros

**CON Data Leakage veríamos:**
- Ganancia de validación ALTA (> 50M o ratio val/train > 0.5)
- Alta correlación entre el score RF y el target en validación
- Las features RF tienen poder predictivo incluso sin entrenar un modelo adicional

### Próximos pasos:
1. Si hay leakage: revisar cómo se filtran los datos en el método `fit()` del transformer
2. Verificar que NO se use información del mes de validación al crear las features
3. Verificar que el Random Forest interno se entrena SOLO con training_months
